# Modeling spiking neural networks with Brian

In [ ]:
# If Brian2 is not yet installed, the following will install it
# (otherwise it will print a number of "Requirement already satisfied" lines)
!pip install brian2

Let's first import "everything" from the Brian 2 package.

This also provides access to the scientific computing package numpy (imported as np), and to the package pyplot from the plotting library matplotlib (imported as plt).

We also ask the notebook to include plots directly in the notebook (instead of showing them in a separate window). Note that lines starting with % are specific commands for the jupyter notebook, they won't work in a Python script, for example.

We also switch off Brian's "code generation" mechanism, which improves the performance for complex models by generating/compiling/executing C++ code "behind the scenes". For the simple models that we are covering in this tutorial, this is not necessary – it even slows down things due to the need for compilation.

In [ ]:
from brian2 import *
prefs.codegen.target = 'numpy'

In [ ]:
%matplotlib inline
%xmode minimal

Exception reporting mode: Minimal


## Neurons

Let's create a group of integrate-and-fire neurons. We use a simplified model where the "membrane potential" is an abstract, dimensionless value -- it's resting potential is 0, and the threshold to elicit a spike is 1. We also add a noise term (`xi` is a builtin variable provided by Brian), ignore the `tau**-0.5`, we need this for correct dimensions of the noise term because of the way it scales with time.

In [ ]:
start_scope()
tau = 10*ms
N = 3
E_L = -70*mV
V_threshold = -50*mV
C = 100*pF
g_L = 10*nS
tau = C/g_L
sigma_noise = 10*mV
neurons = NeuronGroup(N, '''dV/dt = (g_L*(E_L-V) + I_stim)/C + sigma_noise*xi*tau**-0.5: volt
                            I_stim : amp (constant)''',
                      threshold='V>V_threshold', reset='V=E_L', method='euler')
# Initialize values
neurons.V = '(V_threshold - E_L)*0.5*rand() + E_L'
# The stimulus is now between 0.2nA and 0.8nA
neurons.I_stim = '(0.2 + 0.6*i/(N-1))*nA'

# record membrane potential and spikes
v_mon = StateMonitor(neurons, 'V', record=True)
spike_mon = SpikeMonitor(neurons)

# run simulation
run(100*ms)

We could plot things directly by calling `plt.plot`, but by calling `plt.subplots` first we can easily arrange things in subplots.

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(spike_mon.t/ms, spike_mon.i, '|')
ax[1].plot(v_mon.t/ms, v_mon.V.T/mV);
print(spike_mon.count[:])

Typically, neurons in sensory cortices are described as having a "preferred stimulus feature", e.g. cells in the primary visual cortex are often direction selective and have a preferred direction. Let us model this in the simplest way possible by creating a population of neurons where each neuron has its preferred direction $\Theta$ (`theta`), and where the input current strength is a Gaussian function of the difference between the presented stimulus and the preferred stimulus of each cell. Note that for simplicity, the code does not take care of the periodic nature of the stimulus, i.e. that $0$ is the same as $2\pi$.

In [ ]:
start_scope()
tau = 10*ms
sigma_noise = 10*mV
theta_stim = 0    # stimulus
sigma_stim = pi/8  # stimulation width
amp_stim = 0.4*nA      # stimulus amplitude
N = 400
neurons = NeuronGroup(N, '''dV/dt = (g_L*(E_L-V) + I_stim)/C + sigma_noise*xi*tau**-0.5: volt
                            I_stim = amp_stim*exp(-(theta - theta_stim)**2/(2*sigma_stim)**2) : amp
                            theta : 1 (constant)''',
                      threshold='V>V_threshold', reset='V=E_L', method='euler')
# Initialize values
neurons.V = '(V_threshold - E_L)*0.5*rand() + E_L'
neurons.theta = '-pi + 2*pi*i/N'

# record spikes
spike_mon = SpikeMonitor(neurons)

# run simulation
run(100*ms)

In [ ]:
fig, ax = plt.subplots()
ax.plot(spike_mon.t/ms, spike_mon.i, '|');
# TODO: How could we show the preferred direction on the y axis (instead of the neuron index)
# TODO: How could we plot a "population tuning curve" (number of spikes over preferred stimulus)?

## Synapses
Let us know connect our neurons with synapses. For now, the synapses are connected in a uniform fashion amongst each other.

In [ ]:
start_scope()
tau = 10*ms
sigma_noise = 10*mV
theta_stim = 0.; sigma_stim = pi/8; amp_stim = 0.4*nA  # stimulus parameters
tau_e = 50*ms
N_E = 400

# excitatory neurons
exc_neurons = NeuronGroup(N_E, '''dV/dt = (g_L*(E_L-V) + I_stim + I_ee)/C + sigma_noise*xi*tau**-0.5: volt
                                  dI_ee/dt = -I_ee/tau_e : amp
                                  I_stim = amp_stim*exp(-(theta - theta_stim)**2/(2*sigma_stim)**2) : amp
                                  theta : 1 (constant)''',
                          threshold='V>V_threshold', reset='V=E_L', method='euler')
exc_neurons.V = '(V_threshold - E_L)*0.5*rand() + E_L'
exc_neurons.theta = '-pi + 2*pi*i/N_E'

# recurrent excitatory synapses
E_to_E = Synapses(exc_neurons, exc_neurons, 'w: amp', on_pre='I_ee += w')
E_to_E.connect(p=0.5)
E_to_E.w = '40*pA/N_E'

# Monitor neurons
exc_mon = SpikeMonitor(exc_neurons)

# run simulation
run(1000*ms, report='text')
# TODO: Switch off the stimulus and run for another 500ms
# TODO: Can we get sustained activity by increasing the weight?

Starting simulation at t=0. s for a duration of 1. s
1. s (100%) simulated in 3s


In [ ]:
fig, ax = plt.subplots()
ax.plot(exc_mon.t/ms, exc_mon.i, '|');

## Full model
We now add an inhibitory population of neurons which are not activated by the external stimulus, but only from the excitatory population. The inhibitory cells are connected unspecifically to the excitatory cells. The example uses a lot more code now, but all the components are very similar to what we have used before. Our network is now a (crude) model of working memory circuits in the brain: a stimulus will drive the network into a state that sustains the activity for a while after the stimulus is switched off.

In [ ]:
start_scope()
tau = 10*ms
sigma_noise = 10*mV
theta_stim = 0; sigma_stim = pi/8; amp_stim = 0.4*nA  # stimulus parameters
tau_e = 50*ms; tau_i = 5*ms
N_E = 400; N_I = 100

# excitatory neurons
exc_neurons = NeuronGroup(N_E, '''dV/dt = (g_L*(E_L-V) + I_ee + I_ei + I_stim)/C + sigma_noise*xi*tau**-0.5: volt
                                  dI_ee/dt = -I_ee/tau_e : amp
                                  dI_ei/dt = -I_ei/tau_i : amp
                                  I_stim = amp_stim*exp(-(theta - theta_stim)**2/(2*sigma_stim)**2) : amp
                                  theta : 1 (constant)''',
                          threshold='V>V_threshold', reset='V=E_L', method='euler')
exc_neurons.V = '(V_threshold - E_L)*0.5*rand() + E_L'
exc_neurons.theta = '-pi + 2*pi*i/N_E'

# recurrent excitatory synapses
E_to_E = Synapses(exc_neurons, exc_neurons, 'w: amp', on_pre='I_ee += w')
E_to_E.connect(p=0.5)
sigma_struct = pi/8
J_global = 0.13*pA; J_selective = 2.75*pA
E_to_E.w = 'J_global + J_selective*exp(-(theta - theta_stim)**2/(2*sigma_stim)**2)'

# inhibitory neurons
inh_neurons = NeuronGroup(N_I, '''dV/dt = (g_L*(E_L-V) + I_ie + I_ii)/C + sigma_noise*xi*tau**-0.5: volt
                                  dI_ie/dt = -I_ie/tau_e : amp
                                  dI_ii/dt = -I_ii/tau_i : amp''',
                          threshold='V>V_threshold', reset='V=E_L', method='euler')
inh_neurons.V = '(V_threshold - E_L)*0.5*rand() + E_L'

# remaining recurrent synapses
w_ei = 5*nA/N_I; w_ie = 0.2*nA/N_E; w_ii = 0.2*nA/N_I
I_to_I = Synapses(inh_neurons, inh_neurons, on_pre='I_ii -= w_ii')
I_to_I.connect(p=0.5)

I_to_E = Synapses(inh_neurons, exc_neurons, on_pre='I_ei -= w_ei')
I_to_E.connect(p=0.5)

E_to_I = Synapses(exc_neurons, inh_neurons, on_pre='I_ie += w_ie')
E_to_I.connect(p=0.5)

# Monitor all neurons
exc_mon = SpikeMonitor(exc_neurons)
inh_mon = SpikeMonitor(inh_neurons)

# run the simulation
run(500*ms, report='text')

# TODO: Change the stimulation protocol:
#       What does activity look without stimulation (before and after stimulation)?

In [ ]:
fig, ax = plt.subplots(2, 1, gridspec_kw={'height_ratios': [1, 4]}, sharex=True)
ax[0].plot(inh_mon.t/ms, inh_mon.i, '|');
ax[0].set(title='inhibitory neurons', ylabel='index')
ax[1].plot(exc_mon.t/ms, exc_neurons.theta[exc_mon.i], '|');
ax[1].set(title='excitatory_neurons', ylabel='preferred stim', xlabel='time (ms)')
fig.tight_layout()